In [ ]:
import requests
from io import BytesIO

In [ ]:
coll = []
notInBpl = []
 
while not complete:
  data = requests.get(f"https://collections.leventhalmap.org/search.json?per_page=100&search_field=all_fields&q=&search_field=all_fields&page={page}")
  thisPageResult = data.json()
  allPages = thisPageResult['response']['pages']['total_pages']
  print(f'💬 processing page {page}/{allPages}')
 
  for record in thisPageResult['response']['docs']:
    
    manifest = f'https://www.digitalcommonwealth.org/search/{record["id"]}/manifest'
    iiifResponse = requests.get(manifest)
    where = record['physical_location_ssim'][0]
    
    # try:
    if where == "Boston Public Library":
    
      w = iiifResponse.json()['sequences'][0]['canvases'][0]['width']
      h = iiifResponse.json()['sequences'][0]['canvases'][0]['height']
      ar = h/w
 
      coll.append(
        {
          'id': record["id"],
          'w': w,
          'h': h,
          'ar': ar,
          'extent': record["extent_tsi"]
        }
      )
 
      size = 300, 300
      print(record['id'])
      imgUrl = iiifResponse.json()['thumbnail']['@id']
      print(imgUrl)
      imgResponse = requests.get(imgUrl)
      print(imgResponse)
      with Image.open(BytesIO(imgResponse.content)) as im:
        im.thumbnail(size)
        im.save(f"{record['id']}", "PNG")
 
    else:
      notInBpl.append(
        {
          'id': record["id"],
          'w': w,
          'h': h,
          'ar': ar,
          'extent': record["extent_tsi"]
        }
      )
 
    # except Exception as err:
    #   print(f"‼️ {err} on Record {record['id']}")
    
    complete = True if thisPageResult['response']['pages']['last_page?'] else False
    page = thisPageResult['response']['pages']['next_page']
  dfColl = pd.DataFrame(coll)
  dfColl.to_csv('spinner/collections.csv', index=False)
  dfnotInBpl = pd.DataFrame(notInBpl)
  dfnotInBpl.to_csv('spinner/notInBpl.csv', index=False)
  print(f"💾 saved through page {page}/{allPages}!")